In [28]:
import correlation_matrix as co
import numpy as np

In [29]:
data_file = open('historical_factors.csv', 'r')
matrix = np.loadtxt(data_file, skiprows=1,usecols=(2,3,4,5,6,7,8,9,10))
cor = co.Cor_matrix(matrix)
factor = np.array(cor[0,1:])[0]
print(factor)

[  3.73e-01   2.33e-01  -5.47e-01   3.17e-01  -3.61e-16   6.54e-01   6.12e-02  -4.43e-02]


In [32]:
data_file = open('historical_factors.csv', 'r')
lines = data_file.readlines()
print( 'Country \t Cal \t Bib \t Dif')
for line in lines[1:]:
    words = line.split('\t');
    name = words[0] + '_' + words[1]
    weight = words[3:]
    weight = [float(i) for i in weight]
    score = co.Compute_Score(weight,factor)
    cal_sc = round((score+4.32698482534)/(14.6802594247+4.32698482534)*30)
    print( name, '\t', cal_sc, '\t', words[2],'\t', cal_sc-float(words[2])  )

Country 	 Cal 	 Bib 	 Dif
Belarus_2015 	 13.0 	 0.0 	 13.0
Germany_2015 	 5.0 	 0.0 	 5.0
Japan_2015 	 7.0 	 0.0 	 7.0
Armenia_2015 	 15.0 	 0.0 	 15.0
Belgium_2015 	 0.0 	 0.0 	 0.0
Bulgar_2015 	 6.0 	 0.0 	 6.0
Canada_2015 	 0.0 	 0.0 	 0.0
Czech-R_2015 	 0.0 	 0.0 	 0.0
Finland_2015 	 6.0 	 0.0 	 6.0
Hungary_2015 	 2.0 	 0.0 	 2.0
Kazak_2015 	 9.0 	 0.0 	 9.0
Lithua_2015 	 9.0 	 0.0 	 9.0
Mexico_2015 	 5.0 	 0.0 	 5.0
Nether_2015 	 9.0 	 0.0 	 9.0
Saudi-A_2015 	 25.0 	 0.0 	 25.0
Slovak_2015 	 3.0 	 0.0 	 3.0
Spain_2015 	 3.0 	 0.0 	 3.0
Ukraine_2015 	 9.0 	 0.0 	 9.0
UAE_2015 	 24.0 	 0.0 	 24.0
Sweden_2015 	 4.0 	 0.0 	 4.0
Switz_2015 	 3.0 	 0.0 	 3.0
Indones_2015 	 8.0 	 0.0 	 8.0
Algeria_2015 	 17.0 	 0.0 	 17.0
Romania_2015 	 3.0 	 0.0 	 3.0
Sweden_1946 	 18.0 	 1.0 	 17.0
Switz_1946 	 15.0 	 1.0 	 14.0
Indones_1965 	 14.0 	 1.0 	 13.0
Algeria_1983 	 22.0 	 1.0 	 21.0
Romania_1985 	 13.0 	 1.0 	 12.0
Austral_1961 	 9.0 	 2.0 	 7.0
Egypt_1965 	 22.0 	 2.0 	 20.0
Libya_1970 	 25